In [35]:
import sys
sys.path.append('..')

from sqlalchemy import create_engine, text
import pandas as pd
import geopandas as gpd
from shapely.ops import unary_union

from config import RUTA_UNIDAD_ONE_DRIVE
from config import RUTA_LOCAL_ONE_DRIVE
from config import POSTGRES_UTEA

In [37]:
# conexion a la base de datos
def obtener_engine():
    return create_engine(
        f"postgresql+psycopg2://{POSTGRES_UTEA['USER']}:{POSTGRES_UTEA['PASSWORD']}@{POSTGRES_UTEA['HOST']}:{POSTGRES_UTEA['PORT']}/{POSTGRES_UTEA['DATABASE']}"
    )

# selecciona los recorridos sin asignacion (idd = 0)
def get_recorridos_sin_asignacion():
    engine = obtener_engine()
    try:
        query = """
            SELECT * FROM drones_control_bio.recorridos_lib where idd = 0
        """
        gdf = gpd.read_postgis(query, engine, geom_col='geom')
        return gdf
    except Exception as e:
        print(f"❌ Error al obtener toda la capa de recorridos de control biologico: {e}")
        return gpd.GeoDataFrame()
    return None

# selecciona lotes de parte diario liberados (id_labor null y idd not null)
def obtener_partes_sin_id_labor():
    engine = obtener_engine()
    try:
        query = """
            SELECT *
            FROM drones_control_bio.parte_diario_ctrl_bio
            WHERE id_labor is null and idd is not null
        """
        gdf = gpd.read_postgis(query, engine, geom_col='geom')
        return gdf
    except Exception as e:
        print(f"❌ Error al consultar unidades sin labor: {e}")
        return gdf.GeoDataFrame()
    return None

# retorna una lista de codigo de propiedades, propiedades que intersectan con algun recorrido
def get_lista_pros_intersect_recorridos(gdf_lotes, gdf_recorridos):
    # interseccion de lotes y recorridos, se obtienen todas las propiedades que tengan por lo menos un recorrido
    gdf_intersect = gpd.overlay(gdf_parte_diario, gdf_recorridos, how="intersection", keep_geom_type=False)
    # solo se queda con la columna de codigos de propiedades intersectadas
    gdf_intersect = gdf_intersect[['unidad_01']].copy()
    # eliminar duplicados (set), y conversion a lista (list)
    list_cod_props = list(set(gdf_intersect['unidad_01']))
    # ordena la lista
    list_cod_props.sort()
    return list_cod_props

def get_buffer_16m(gdf_recorridos):
    # buffer de 16m de los recorridos
    gdf_buffer_16m = gdf_recorridos_prop.geometry.buffer(16)
    # union o disolve de los buffer, retorna una objeto poligono
    gdf_buffer_16m_disolve = unary_union(gdf_buffer_16m)
    # convertir el objeto poligono en gdf
    gdf_buffer_16m = gpd.GeoDataFrame(geometry=[gdf_buffer_16m_disolve], crs=gdf_buffer_16m.crs)
    return gdf_buffer_16m

In [38]:
gdf_recorridos = get_recorridos_sin_asignacion()

In [39]:
gdf_parte_diario = obtener_partes_sin_id_labor()

In [ ]:
# lista de codigos de propiedades con algun recorrido 
lista_cod_props = get_lista_pros_intersect_recorridos(gdf_parte_diario, gdf_recorridos)

In [ ]:
lista_cod_props

In [ ]:
# seleccion de un propiedad
cod_prop = 2250

In [ ]:
gdf_lotes_prop = gdf_parte_diario[gdf_parte_diario['unidad_01'] == cod_prop]
gdf_lotes_prop.plot()

In [ ]:
# validar que todos los lotes tienen estado = 'EJECUTADO'
estado = list(set(gdf_lotes_prop["estado"]))
if len(estado) == 1 and estado[0] == 'EJECUTADO':
    print('Todos los lotes de la propiedad estan ejecutados')
else:
    print('Propiedad tiene lotes sin ejecutar')

In [ ]:
# seleccion de recorridos que intersectar con la propiedad
gdf_recorridos_prop = gpd.overlay(gdf_recorridos, gdf_lotes_prop, how="intersection", keep_geom_type=False)

In [ ]:
# buffer de 16m de recorridos
gdf_buffer_16m = get_buffer_16m(gdf_recorridos_prop)

In [ ]:
# recorte de buffer_16m con lotes de propiedad seleccionada
gdf_clip_buffer_lotes = gpd.clip(gdf_lotes_prop, gdf_buffer_16m)

In [ ]:
# calcular cantidad de lotes en propiedad y el area cortada
cantidad_lotes_prop = len(gdf_lotes_prop)
cantidad_lotes_clip = len(gdf_clip_buffer_lotes)

In [ ]:
# comparacion de cantidad de lotes
if cantidad_lotes_prop == cantidad_lotes_clip:
    print('Todos los lotes de la propiedad tiene recorrido')
else:
    print('No todos los lotes de la propiedad tiene recorrido')

In [ ]:
# agregar campo area_2 para calcular area de cobertura
gdf_clip_buffer_lotes["area_2"] = gdf_clip_buffer_lotes.geometry.area / 10000

In [ ]:
# calcular porcentaje de ;l
gdf_clip_buffer_lotes['porcen'] = gdf_clip_buffer_lotes['area_2'] / gdf_clip_buffer_lotes['area']

In [ ]:
gdf_clip_buffer_lotes.plot()